In [18]:
from sklearn import datasets
import numpy as np

iris = datasets.load_diabetes()

X = iris.data
y = iris.target

In [19]:
iris.feature_names

['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
import xgboost as xgb

model = xgb.XGBRegressor()
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [22]:
model.score(X_test, y_test)

0.3675149756138415

In [36]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/predict', methods=['GET'])
def predict():
    features = [
        float(request.args.get('age')),
        float(request.args.get('sex')),
        float(request.args.get('bmi')),
        float(request.args.get('blood_pressure')),
        float(request.args.get('s1')),
        float(request.args.get('s2')),
        float(request.args.get('s3')),
        float(request.args.get('s4')),
        float(request.args.get('s5')),
        float(request.args.get('s6'))
    ]
    
    features = np.array(features).reshape(1, -1)
    prediction = model.predict(features)
    print(prediction)

    return jsonify({'prediction': float(prediction[0]), 'model': 'XGBoost'})


if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [11/Feb/2025 16:24:55] "GET /predict?age=0.070769&sex=0.050680&bmi=0.012117&blood_pressure=0.056301&s1=0.034206&s2=0.049416&s3=-0.039719&s4=0.034309&s5=0.027364&s6=-0.001078 HTTP/1.1" 200 -


[159.922]


127.0.0.1 - - [11/Feb/2025 16:24:55] "GET /predict?age=-0.009147&sex=0.050680&bmi=-0.018062&blood_pressure=-0.033213&s1=-0.020832&s2=0.012152&s3=-0.072854&s4=0.071210&s5=0.000272&s6=0.019633 HTTP/1.1" 200 -


[154.13858]


...

127.0.0.1 - - [11/Feb/2025 16:24:59] "GET /predict?age=-0.012780&sex=-0.044642&bmi=-0.023451&blood_pressure=-0.040099&s1=-0.016704&s2=0.004636&s3=-0.017629&s4=-0.002592&s5=-0.038460&s6=-0.038357 HTTP/1.1" 200 -


[76.25413]


127.0.0.1 - - [11/Feb/2025 16:24:59] "GET /predict?age=-0.092695&sex=-0.044642&bmi=0.028284&blood_pressure=-0.015999&s1=0.036958&s2=0.024991&s3=0.056003&s4=-0.039493&s5=-0.005142&s6=-0.001078 HTTP/1.1" 200 -


[284.2587]


127.0.0.1 - - [11/Feb/2025 16:25:15] "GET / HTTP/1.1" 404 -


## Question 2 - Using ngrok

In [32]:
import requests
from collections import Counter

urls = [
    "https://3312-2a01-e34-ec54-b5f0-1ed-2339-12c4-f846.ngrok-free.app",
    "https://54bb-2a04-cec2-a-5ee3-9cdf-d03e-8cb9-1039.ngrok-free.app",
    "https://8245-2a01-cb00-1401-e900-3552-1c20-acea-899d.ngrok-free.app"
    # ""
]


def get_predictions(urls):
    predictions = []
    
    for url in urls:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if 'prediction' in data:
            predictions.append(data['prediction'])

    return predictions

predictions = get_predictions(urls)
mean_prediction = sum(predictions) / len(predictions) * 1.0

HTTPError: 404 Client Error: Not Found for url: https://3312-2a01-e34-ec54-b5f0-1ed-2339-12c4-f846.ngrok-free.app/

Rom1

In [11]:
import requests
from collections import Counter

def get_predictions(urls):
    predictions = []
    
    for url in urls:
        try:
            response = requests.get(url)
            response.raise_for_status()  # Raise an error for bad responses (4xx, 5xx)
            data = response.json()
            if 'prediction' in data:
                predictions.append(data['prediction'])
                print(data)
        except requests.RequestException as e:
            print(f"Error with request to {url}: {e}")
    
    if predictions:
        return sum(predictions)/len(predictions)
    else:
        return None


arguments = [
    "/predict?age=59&sex=2&bmi=32.1&blood_pressure=101&s1=157&s2=93.2&s3=38&s4=4&s5=4.8598&s6=87",
    "/predict?age=48&sex=1&bmi=21.6&blood_pressure=87&s1=183&s2=103.2&s3=70&s4=3&s5=3.8918&s6=69",
    "/predict?age=72&sex=2&bmi=30.5&blood_pressure=93&s1=156&s2=93.6&s3=41&s4=4&s5=4.6728&s6=85",
    "/predict?age=24&sex=1&bmi=25.3&blood_pressure=84&s1=198&s2=131.4&s3=40&s4=5&s5=4.8903&s6=89",
    "/predict?age=50&sex=1&bmi=23.0&blood_pressure=101&s1=192&s2=125.4&s3=52&s4=4&s5=4.2905&s6=80",
    "/predict?age=23&sex=1&bmi=22.6&blood_pressure=89&s1=139&s2=64.8&s3=61&s4=2&s5=4.1897&s6=68",
    "/predict?age=36&sex=2&bmi=22.0&blood_pressure=90&s1=160&s2=99.6&s3=50&s4=3&s5=3.9512&s6=82",
    "/predict?age=66&sex=2&bmi=26.2&blood_pressure=114&s1=255&s2=185&s3=56&s4=4.55&s5=4.2485&s6=92",
    "/predict?age=60&sex=2&bmi=32.1&blood_pressure=83&s1=179&s2=119.4&s3=42&s4=4&s5=4.4773&s6=94",
    "/predict?age=29&sex=1&bmi=30.0&blood_pressure=85&s1=180&s2=93.4&s3=43&s4=4&s5=5.3845&s6=88"
]

for arg in arguments :
    url1 = "https://54bb-2a04-cec2-a-5ee3-9cdf-d03e-8cb9-1039.ngrok-free.app" + arg
    url2 = "https://3312-2a01-e34-ec54-b5f0-1ed-2339-12c4-f846.ngrok-free.app" + arg
    url3 = "https://3312-2a01-e34-ec54-b5f0-1ed-2339-12c4-f846.ngrok-free.app" + arg
    url4 = "" + arg

    result = get_predictions([url1, url2])

    print(arg)
    if result:
        print(f"Most common prediction: {str(result)}")
    else:
        print("No valid predictions received.")

{'model': 'RandomForestClassifier', 'prediction': 254.7}
{'model': 'KNN', 'prediction': 186.33333333333334}
/predict?age=59&sex=2&bmi=32.1&blood_pressure=101&s1=157&s2=93.2&s3=38&s4=4&s5=4.8598&s6=87
Most common prediction: 220.51666666666665
{'model': 'RandomForestClassifier', 'prediction': 254.7}
{'model': 'KNN', 'prediction': 173.66666666666666}
/predict?age=48&sex=1&bmi=21.6&blood_pressure=87&s1=183&s2=103.2&s3=70&s4=3&s5=3.8918&s6=69
Most common prediction: 214.18333333333334
{'model': 'RandomForestClassifier', 'prediction': 254.7}
{'model': 'KNN', 'prediction': 173.66666666666666}
/predict?age=72&sex=2&bmi=30.5&blood_pressure=93&s1=156&s2=93.6&s3=41&s4=4&s5=4.6728&s6=85
Most common prediction: 214.18333333333334
{'model': 'RandomForestClassifier', 'prediction': 254.7}
{'model': 'KNN', 'prediction': 173.66666666666666}
/predict?age=24&sex=1&bmi=25.3&blood_pressure=84&s1=198&s2=131.4&s3=40&s4=5&s5=4.8903&s6=89
Most common prediction: 214.18333333333334
{'model': 'RandomForestClassi

## Q3 - Weighting system

In [37]:
import requests

weights = {
    "https://3312-2a01-e34-ec54-b5f0-1ed-2339-12c4-f846.ngrok-free.app": 1.0,
    "https://54bb-2a04-cec2-a-5ee3-9cdf-d03e-8cb9-1039.ngrok-free.app": 1.0,
    "https://8245-2a01-cb00-1401-e900-3552-1c20-acea-899d.ngrok-free.app": 1.0
}
def get_predictions(urls):
    predictions = {}

    for url in urls:
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            if 'prediction' in data:
                predictions[url] = data['prediction']
        except requests.RequestException as e:
            print(f"Error fetching prediction from {url}: {e}")

    return predictions

def aggregate_weighted_prediction(predictions, weights):
    total_weight = sum(weights[url] for url in predictions)
    weighted_prediction = sum(weights[url] * predictions[url] for url in predictions) / total_weight
    return weighted_prediction

def update_weights(predictions, weights, top_n=1):
    """ Update weights based on the top n best-performing models. """
    weight_adjust_rate = 0.001

    # Sort predictions by their values in descending order
    sorted_predictions = sorted(predictions.items(), key=lambda item: item[1], reverse=True)

    # Check if there are enough predictions to select top_n
    if len(sorted_predictions) < top_n:
        print(f"Not enough predictions to select top {top_n}. Adjusting top_n to {len(sorted_predictions)}.")
        top_n = len(sorted_predictions)

    # Identify the top n predictions
    top_n_urls = {url for url, _ in sorted_predictions[:top_n]}

    # Adjust weights based on ranking
    for url, prediction in predictions.items():
        u

        if url in top_n_urls:
            weights[url] = min(weights[url] + weight_adjust_rate, 1.0)  # Increase weight for top n
        else:
            weights[url] = max(weights[url] - weight_adjust_rate, 0.0)  # Decrease weight for others

In [38]:
arguments = [
    "/predict?age=0.070769&sex=0.050680&bmi=0.012117&blood_pressure=0.056301&s1=0.034206&s2=0.049416&s3=-0.039719&s4=0.034309&s5=0.027364&s6=-0.001078",
    "/predict?age=-0.009147&sex=0.050680&bmi=-0.018062&blood_pressure=-0.033213&s1=-0.020832&s2=0.012152&s3=-0.072854&s4=0.071210&s5=0.000272&s6=0.019633",
    "/predict?age=0.005383&sex=-0.044642&bmi=0.049840&blood_pressure=0.097615&s1=-0.015328&s2=-0.016345&s3=-0.006584&s4=-0.002592&s5=0.017036&s6=-0.013504",
    "/predict?age=-0.027310&sex=-0.044642&bmi=-0.035307&blood_pressure=-0.029770&s1=-0.056607&s2=-0.058620&s3=0.030232&s4=-0.039493&s5=-0.049872&s6=-0.129483",
    "/predict?age=-0.023677&sex=-0.044642&bmi=-0.065486&blood_pressure=-0.081413&s1=-0.038720&s2=-0.053610&s3=0.059685&s4=-0.076395&s5=-0.037129&s6=-0.042499",
    "/predict?age=-0.096328&sex=-0.044642&bmi=-0.076264&blood_pressure=-0.043542&s1=-0.045599&s2=-0.034821&s3=0.008142&s4=-0.039493&s5=-0.059471&s6=-0.083920",
    "/predict?age=0.005383&sex=0.050680&bmi=0.030440&blood_pressure=0.083844&s1=-0.037344&s2=-0.047347&s3=0.015505&s4=-0.039493&s5=0.008641&s6=0.015491",
    "/predict?age=0.030811&sex=-0.044642&bmi=-0.020218&blood_pressure=-0.005670&s1=-0.004321&s2=-0.029497&s3=0.078093&s4=-0.039493&s5=-0.010903&s6=-0.001078",
    "/predict?age=-0.012780&sex=-0.044642&bmi=-0.023451&blood_pressure=-0.040099&s1=-0.016704&s2=0.004636&s3=-0.017629&s4=-0.002592&s5=-0.038460&s6=-0.038357",
    "/predict?age=-0.092695&sex=-0.044642&bmi=0.028284&blood_pressure=-0.015999&s1=0.036958&s2=0.024991&s3=0.056003&s4=-0.039493&s5=-0.005142&s6=-0.001078"
]
for argument in arguments:
    new_urls = [url + argument for url in urls]
    print(new_urls)
    predictions = get_predictions(new_urls)

    update_weights(predictions, weights)

    weighted_mean_prediction = aggregate_weighted_prediction(predictions, weights)
    print("Weighted Mean Prediction:", weighted_mean_prediction)
    print("Updated Weights:", weights)

['https://3312-2a01-e34-ec54-b5f0-1ed-2339-12c4-f846.ngrok-free.app/predict?age=0.070769&sex=0.050680&bmi=0.012117&blood_pressure=0.056301&s1=0.034206&s2=0.049416&s3=-0.039719&s4=0.034309&s5=0.027364&s6=-0.001078', 'https://54bb-2a04-cec2-a-5ee3-9cdf-d03e-8cb9-1039.ngrok-free.app/predict?age=0.070769&sex=0.050680&bmi=0.012117&blood_pressure=0.056301&s1=0.034206&s2=0.049416&s3=-0.039719&s4=0.034309&s5=0.027364&s6=-0.001078', 'https://8245-2a01-cb00-1401-e900-3552-1c20-acea-899d.ngrok-free.app/predict?age=0.070769&sex=0.050680&bmi=0.012117&blood_pressure=0.056301&s1=0.034206&s2=0.049416&s3=-0.039719&s4=0.034309&s5=0.027364&s6=-0.001078']


KeyError: 'https://3312-2a01-e34-ec54-b5f0-1ed-2339-12c4-f846.ngrok-free.app/predict?age=0.070769&sex=0.050680&bmi=0.012117&blood_pressure=0.056301&s1=0.034206&s2=0.049416&s3=-0.039719&s4=0.034309&s5=0.027364&s6=-0.001078'

In [39]:
weights

{'https://3312-2a01-e34-ec54-b5f0-1ed-2339-12c4-f846.ngrok-free.app': 1.0,
 'https://54bb-2a04-cec2-a-5ee3-9cdf-d03e-8cb9-1039.ngrok-free.app': 1.0,
 'https://8245-2a01-cb00-1401-e900-3552-1c20-acea-899d.ngrok-free.app': 1.0}

Rom1

In [ ]:
import requests
from collections import Counter
import numpy as np

def get_predictions(urls):
    predictions = []
    
    for url in urls:
        try:
            response = requests.get(url)
            response.raise_for_status()  
            data = response.json()
            if 'prediction' in data:
                predictions.append(data['prediction'])
        except requests.RequestException as e:
            print(f"Error with request to {url}: {e}")
    return predictions


# Initialize model weights
weights = np.array([1.0, 1.0, 1.0, 1.0])

arguments = [
    "/predict?age=0.070769&sex=0.050680&bmi=0.012117&blood_pressure=0.056301&s1=0.034206&s2=0.049416&s3=-0.039719&s4=0.034309&s5=0.027364&s6=-0.001078",
    "/predict?age=-0.009147&sex=0.050680&bmi=-0.018062&blood_pressure=-0.033213&s1=-0.020832&s2=0.012152&s3=-0.072854&s4=0.071210&s5=0.000272&s6=0.019633",
    "/predict?age=0.005383&sex=-0.044642&bmi=0.049840&blood_pressure=0.097615&s1=-0.015328&s2=-0.016345&s3=-0.006584&s4=-0.002592&s5=0.017036&s6=-0.013504",
    "/predict?age=-0.027310&sex=-0.044642&bmi=-0.035307&blood_pressure=-0.029770&s1=-0.056607&s2=-0.058620&s3=0.030232&s4=-0.039493&s5=-0.049872&s6=-0.129483",
    "/predict?age=-0.023677&sex=-0.044642&bmi=-0.065486&blood_pressure=-0.081413&s1=-0.038720&s2=-0.053610&s3=0.059685&s4=-0.076395&s5=-0.037129&s6=-0.042499",
    "/predict?age=-0.096328&sex=-0.044642&bmi=-0.076264&blood_pressure=-0.043542&s1=-0.045599&s2=-0.034821&s3=0.008142&s4=-0.039493&s5=-0.059471&s6=-0.083920",
    "/predict?age=0.005383&sex=0.050680&bmi=0.030440&blood_pressure=0.083844&s1=-0.037344&s2=-0.047347&s3=0.015505&s4=-0.039493&s5=0.008641&s6=0.015491",
    "/predict?age=0.030811&sex=-0.044642&bmi=-0.020218&blood_pressure=-0.005670&s1=-0.004321&s2=-0.029497&s3=0.078093&s4=-0.039493&s5=-0.010903&s6=-0.001078",
    "/predict?age=-0.012780&sex=-0.044642&bmi=-0.023451&blood_pressure=-0.040099&s1=-0.016704&s2=0.004636&s3=-0.017629&s4=-0.002592&s5=-0.038460&s6=-0.038357",
    "/predict?age=-0.092695&sex=-0.044642&bmi=0.028284&blood_pressure=-0.015999&s1=0.036958&s2=0.024991&s3=0.056003&s4=-0.039493&s5=-0.005142&s6=-0.001078"
]

# URLs for models
model_urls = [
    "https://8e9c-92-169-192-254.ngrok-free.app",
    "https://54bb-2a04-cec2-a-5ee3-9cdf-d03e-8cb9-1039.ngrok-free.app",
    "https://3312-2a01-e34-ec54-b5f0-1ed-2339-12c4-f846.ngrok-free.app",
    "https://8245-2a01-cb00-1401-e900-3552-1c20-acea-899d.ngrok-free.app"
]

for arg in arguments:
    urls = [url + arg for url in model_urls]
    predictions = get_predictions(urls)

    if predictions:
        predictions = np.array(predictions)

        # Compute weighted mean prediction
        mean_prediction = np.average(predictions, weights=weights)
        print(f"Predictions: {predictions}")
        print(f"Weighted Mean Prediction: {mean_prediction}")

        # Update weights based on error
        errors = np.abs(predictions - mean_prediction)
        new_weights = np.exp(-errors)  # Lower error -> Higher weight

        # Apply smoothing to avoid drastic weight changes
        alpha = 0.2  # Adjust weight update sensitivity
        weights = (1 - alpha) * weights + alpha * new_weights

        # Normalize weights to sum to 1
        weights /= weights.sum()
        
        print(f"Updated model weights: {weights}")
    else:
        print("No valid predictions received.")